<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Ripples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google import colab


warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)



!pip install openpyxl
!pip install XlsxWriter
!pip install --upgrade pyabf
import pyabf


clear_output()


In [2]:
def get_drobox_folder(link, new_filename):
    'Download a folder from dropbox and unzip'
    zipped_file_path = "/content/"+new_filename + ".zip"
    unzipped_file_path = "/content/"+new_filename
    if not( os.path.exists(zipped_file_path)):
        !wget -O $zipped_file_path $link    # download with new name
    # if not( os.path.exists(new_filename_stripped)):
    !echo A | unzip $zipped_file_path -d $unzipped_file_path
    return new_filename




data_source = "https://www.dropbox.com/sh/cz03im6sa98nquz/AAB-WXnlE4jdTp37CiwVPUCLa?dl=0"
data_name = 'rip_data'
file_loc = get_drobox_folder(data_source, data_name)
file_list = []
for (root, dirs, file) in os.walk(file_loc):
    for f in file:
        if ".abf" in f:
            file_list.append(os.path.join(root,f))


clear_output()


In [3]:
####### MATH
def z_trans_2(x):
    x_mean = np.expand_dims(np.mean(x,axis=1),1)
    x_std = np.expand_dims(np.std(x,axis=1),1)
    x_z = (x - x_mean) / x_std
    return x_z

def z_trans(x):
    x_mean = np.mean(x)
    x_std = np.std(x)
    x_z = (x - x_mean) / x_std
    return x_z

def mov_mean(x,stride):
    x_mm = np.convolve(x,np.ones(stride),'same')/stride
    return x_mm

In [4]:
file_list = []

dates_to_analyze = ['2023x07x12','2023x07x14','2023x07x17']
for (rt,dr,files) in os.walk('/content/rip_data/'):
    for f in files:
        for d in dates_to_analyze:
            if d in f and '.abf' in f[-4:]:
                file_list.append(os.path.join(rt,f))

print(len(file_list))

33


In [5]:
def time_bands(abf,sweepNumber=0,channel=0,order = 4):
    fs = abf.sampleRate
    abf.setSweep(sweepNumber=sweepNumber,channel=channel)
    v_trace = abf.sweepY
    time = abf.sweepX

    new_fs = 1000
    ds_factor = np.ceil(fs/new_fs)
    v_trace_1khz = v_trace[::int(ds_factor)]
    time_1khz = time[::int(ds_factor)]

    b, a = scipy.signal.butter(order, [0.7,300], btype='bandpass',fs=new_fs)
    wide_band = scipy.signal.filtfilt(b, a, v_trace_1khz)

    b, a = scipy.signal.butter(order, [150,250], btype='bandpass',fs=new_fs)
    ripple_band = scipy.signal.filtfilt(b, a, wide_band)

    b, a = scipy.signal.butter(order, [15 ,50], btype='bandpass',fs=new_fs)
    slow_gamma_band = scipy.signal.filtfilt(b, a, wide_band)

    b, a = scipy.signal.butter(order, [1 ,20], btype='bandpass',fs=new_fs)
    sharp_wave_band = scipy.signal.filtfilt(b, a, wide_band)

    # b, a = scipy.signal.butter(order, [6 ,12], btype='bandpass',fs=new_fs)
    # theta_band = scipy.signal.filtfilt(b, a, wide_band)



    freq_bands = {'fs':fs,
                  'time_1khz':time_1khz,
                  'wide_band':wide_band,
                  'sharp_wave_band':sharp_wave_band,
                  'ripple_band':ripple_band,
                  'slow_gamma_band':slow_gamma_band,
                  }

    for k in freq_bands.keys():
        if 'band' in k:
                amplitude_envelope = np.abs(scipy.signal.hilbert(freq_bands[k]))
                freq_bands[k] = {'lfp': freq_bands[k], 'hilb_env':amplitude_envelope}

    return freq_bands

In [6]:

# col_list = ['fs','time_1khz','wide_band','sharp_wave_band','ripple_band','slow_gamma_band']
# rip_df = pd.DataFrame(index=file_list,columns=col_list)
rip_df = pd.DataFrame(index=file_list)

chan_list = ['ch0','ch1']
for c in chan_list:
    rip_df[c]=np.nan
    rip_df[c] = rip_df[c].astype('object')


for f in rip_df.index:
    try:
        abf=pyabf.ABF(f)
        freq_bands = time_bands(abf,sweepNumber=0,channel=0)
        rip_df.at[f,'ch0'] = freq_bands
        freq_bands = time_bands(abf,sweepNumber=0,channel=1)
        rip_df.at[f,'ch1'] = freq_bands
    except: None
display(rip_df.head(20))

,ch0,ch1
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s6x7_0003.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s6x7_0006.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s6x7_0011.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s6x7_0012.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s4x5_0000.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s6x7_0010.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s002_0000.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s6x7_0004.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s6x7_0008.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."
/content/rip_data/2023_07_14/2023x07x14_E3KI_F_P279_s6x7_0007.abf,"{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ...","{'fs': 5000, 'time_1khz': [0.0, 0.001, 0.002, ..."


In [7]:
def inspect_trace(trace,time,line_length=10,start_stop=[],z_scale=3,single_trace_plot_size=[30,2],ax_prev=False,spec_color=False):
    ''' waterfall plot of trace '''
    if len(start_stop) <2: start_stop=[time[0],time[-1]]
    fs = 1/(time[1]-time[0])
    max_t_adj =  np.ceil(start_stop[1]/line_length)*line_length
    pad_width =  int(fs*(max_t_adj) - len(time))
    pad_trace = np.pad(trace,((0,pad_width)),constant_values=(np.nan,np.nan))
    pad_time =  np.pad(time,pad_width,'constant',constant_values=(np.nan,np.nan))

    tics_per_line = int(fs*line_length)
    num_lines = int(len(time)/tics_per_line)+1
    pad_trace_stack = np.reshape(pad_trace,[num_lines,tics_per_line])

    line_time = np.arange(tics_per_line)/fs
    line_time_stack = np.resize(line_time,[num_lines,tics_per_line] )



    y_adjust_const = np.std(trace)*z_scale
    y_adj = y_adjust_const * (np.cumsum(np.ones_like(pad_trace_stack)*-1,axis=0)+1)

    if not ax_prev:
        plt.rcParams["axes.prop_cycle"] = plt.cycler("color", np.flip(plt.cm.viridis(np.linspace(0,1,10)),axis=0))
        fig,ax=plt.subplots(figsize=(single_trace_plot_size[0],single_trace_plot_size[1]*num_lines))
    else: (fig,ax) = ax_prev

    if not spec_color: ax.plot(line_time_stack.T,(pad_trace_stack+y_adj).T)
    else: ax.plot(line_time_stack.T,(pad_trace_stack+y_adj).T,color = spec_color)



    from matplotlib.ticker import MultipleLocator
    ax.yaxis.set_minor_locator(MultipleLocator(z_scale/2))

    return fig,ax

# z_scale = 6
# fig,ax = inspect_trace(z_trans(rip_df.loc[f,'filtered']['slow_gamma_band']['hilb_env']),rip_df.loc[f,'filtered']['time_1khz'],line_length=10,z_scale=z_scale,spec_color='grey')
# ax.set_xlim(0,.6)
# ax.grid(visible=True, which='both',axis='both')
# fig,ax = inspect_trace(z_trans(rip_df.loc[f,'filtered']['ripple_band']['hilb_env']),rip_df.loc[f,'filtered']['time_1khz'],line_length=10,z_scale=z_scale,ax_prev=(fig,ax),spec_color='m')

# plt.show()

In [8]:
ch = 'ch0'
filtered = rip_df.loc[f,ch]

In [9]:
def triggered_inds(trace,fs,sd_thresh= 3,min_dur_ms= 5,peak_wind_ms=20):
    ''' trigger on crossing'''
    trig_bool = z_trans(trace)>sd_thresh
    min_dur_tic = int(min_dur_ms*fs/1000)
    trig_bool = mov_mean(trig_bool,min_dur_tic)==1
    trigs = np.arange(len(trig_bool))[np.diff(trig_bool,prepend=0)==1]

    ''' allign peaks'''
    tick_range = peak_wind_ms/1000*fs
    peak_window_tics = np.arange(0,tick_range,dtype=int)
    trig_alligned = []
    max_tick = len(trace)
    for t in trigs:
        if t+peak_window_tics[0]>=0 and t+peak_window_tics[-1]<=max_tick:
            sub_trace = trace[t+peak_window_tics]
            peak_tick = np.where(sub_trace==np.max(sub_trace))[0][0]
            trig_alligned.append(t+peak_tick-np.min(peak_window_tics))
    trig_alligned=np.array(trig_alligned)

    return trig_alligned


trig_bands = ['ripple_band','sharp_wave_band','slow_gamma_band']
for f in rip_df.index:
    for ch in chan_list:
        filtered = rip_df.loc[f,ch]
        for b in trig_bands:
            trace = filtered[b]['hilb_env']
            fs = filtered['fs']
            trigs = triggered_inds(trace,fs)
            filtered[b]['trigs'] = trigs
        rip_df.at[f,ch] =filtered

In [21]:
### Parse Epochs ###

rip_notes = pd.read_csv('/content/rip_data/Ripples.csv') # .set_index('Slice_Code')
keep_rows = []
for r in rip_notes.index:
    sl = str(rip_notes.loc[r,'Slice_Code'])
    check_slice = any([sl in f for f in file_list])
    if check_slice: keep_rows.append(True)
    else: keep_rows.append(False)
rip_notes = rip_notes[keep_rows]


for r in rip_notes.index:
    sl = rip_notes.loc[r,'Slice_Code']
    number =rip_notes.loc[r,'rec']
    if not np.isnan(number):    rec_str = f"{int(number):04d}"
    else:    rec_str = 'nan'
    slice_code_plus = sl+ '_' + rec_str
    rip_notes.at[r,'slice_code_plus']=slice_code_plus
rip_notes = rip_notes.set_index('slice_code_plus')


for f in rip_df.index:
    f_slice = os.path.basename(f)[:-4]
    if f_slice in rip_notes.index:
        # print(f_slice)
        # print(rip_notes.loc[f_slice,'epoch_0'])
        rip_df.at[f,'epoch_0'] = rip_notes.loc[f_slice,'epoch_0']
        rip_df.at[f,'epoch_1'] = rip_notes.loc[f_slice,'epoch_1']


rip_df_keep = []
for f in rip_df.index:
    f_slice = os.path.basename(f)[:-4]
    rip_df_keep.append(f_slice in rip_notes.index)
print(rip_df_keep)

[False, False, False, False, True, False, False, False, False, False, False, False, True, True, True, True, False, False, False, True, False, True, True, False, False, True, True, True, True, True, False, False, True]


In [ ]:
def get_waveforms_stack(waveform,trigs,fs,window_ms=[-150,150]):
    window_s = np.array(window_ms)/fs
    window_tics = np.arange(window_ms[0]*fs,window_ms[1]*fs,1,dtype=int)
    fig,ax=plt.subplots(1,1, figsize=(6,4))
    wave_snip_list = []
    for t in trigs:
        try: wave_snip_list.append(waveform[t+window_tics])
        except: None
    wave_snip_stack = np.stack(wave_snip_list,axis=-1)
    ax.plot(window_tics/fs,wave_snip_stack,'grey')
    ax.plot(window_tics/fs,np.median(wave_snip_stack,axis=1),'k')
    return wave_snip_stack

In [ ]:


# abf = pyabf.ABF('/content/Bumet_ephys_data/2023x07x06_E3KI_F_P525_s3x4_0000.abf')
# freq_bands = time_bands(abf,sweepNumber=0,channel=0)


def plot_abund_ts(time_1khz,trigs,bin_wid=10):
    fig,ax=plt.subplots(1,figsize=(16,4))
    bins = np.arange(0,np.max(time_1khz)+bin_wid,bin_wid)
    hist,edges = np.histogram(time_1khz[trigs], bins=bins)
    ax.bar(bins[1:],hist/bin_wid,width=bin_wid)
    ax.set_ylabel('SW abundance (Hz)')
    plt.show()
    return fig, ax


# trigs_alligned, fig, ax = get_SW_trig_inds(sharp_wave_band,fs,sw_thresh=-4,to_plot=True)

In [ ]:
# # from scipy import signal





# fs = 10000
# w = 6.
# top_freq = fs/2
# top_freq = 600
# freq = np.linspace(1, top_freq, 100)
# freq = np.logspace(np.log2(10), np.log2(top_freq), num=40, endpoint=True, base=2)
# widths = w*fs / (2*freq*np.pi)
# cwtm = scipy.signal.cwt(in0, scipy.signal.morlet2, widths, w=w)

# cw_abs = np.abs(cwtm)
# cw_mean = np.expand_dims(np.mean(cw_abs,axis=1),-1)
# cw_std = np.expand_dims(np.std(cw_abs,axis=1),-1)
# cw_z = (cw_abs - cw_mean) / cw_std



